In [1]:
import pandas as pd

In [2]:
# === Step 1: Load CSVs ===
orders_df = pd.read_csv("order_details.csv")
stock_df = pd.read_csv("product_stock_levels.csv")


In [3]:
# === Step 2: Handle missing quantities ===
# Impute missing quantities with median per product (or overall median if not enough data)
orders_df['quantity'] = orders_df.groupby('product_id')['quantity'] \
    .transform(lambda x: x.fillna(x.median()))
orders_df['quantity'] = orders_df['quantity'].fillna(orders_df['quantity'].median()).astype(int)

In [4]:
# === Step 3: Parse datetime columns ===
orders_df['order_timestamp'] = pd.to_datetime(orders_df['order_timestamp'])
stock_df['timestamp'] = pd.to_datetime(stock_df['timestamp'])

In [5]:
# === Step 4: Sort stock data for lookup ===
stock_df = stock_df.sort_values(by=['product_id', 'timestamp'])

In [6]:
# === Step 5: Define function to get latest stock level at order time ===
def get_latest_stock(product_id, order_time):
    product_stock = stock_df[(stock_df['product_id'] == product_id) &
                             (stock_df['timestamp'] <= order_time)]
    if not product_stock.empty:
        return product_stock.iloc[-1]['stock_level']
    else:
        return None  # If no prior stock info is found

In [7]:
# === Step 6: Apply enrichment ===
orders_df['stock_level_at_order_time'] = orders_df.apply(
    lambda row: get_latest_stock(row['product_id'], row['order_timestamp']),
    axis=1
)

In [9]:
# === Step 7: Export enriched file ===
orders_df.to_csv("enriched_orders.csv", index=False)
print("enriched_orders.csv has been created.")

enriched_orders.csv has been created.
